滑动窗口（Sliding Windows）算法在TCP 流量控制中有着至关重要的作用。滑动窗口的思想在解决一些 Leetcode 上的题目也非常有效。通常在求一些连续的子数组，子串的问题中有着化繁为简的功效。

下面就针对一些 leetcode上的题目来说明一下使用滑动窗口的套路。滑动窗口的题多半是 medium 和 hard 级别

## 子数组最大平均数 I

leetcode的[643. 子数组最大平均数 I](https://leetcode-cn.com/problems/maximum-average-subarray-i/)，题目如下：
## 

> 给定 n 个整数，找出平均数最大且长度为 k 的连续子数组，并输出该最大平均数。
> 
> 示例 1:
> 
> 输入: [1,12,-5,-6,50,3], k = 4
> 
> 输出: 12.75
> 
> 解释: 最大平均数 (12-5-6+50)/4 = 51/4 = 12.75

题意很明确，例如上面的输入。可以把所有连续的子数组都取出来然后相加，最后再求出最大的平均值。如下图：

![image.png](https://s2.ax1x.com/2020/01/09/lWz9oV.jpg)

也就是求 0->3  1->4 2->5 这三个连续子数组的和，然后取平均值

In [2]:
from typing import *

class Solution:
    def findMaxAverage(self, nums: List[int], k: int) -> float:
        if not nums:
            return 0
        ret = float('-inf')
        for i in range(len(nums) - k):
            r = 0
            for j in range(i, i + k):
                r += nums[j]
            ret = max(r / k, ret)
        return ret

上面的代码使用暴力方式，最外层的的变量 i 几乎要遍历整个数组，其复杂度是O(N)。嵌套的循环也要进行 k 次迭代，其复杂度是 O(K)。综合而言，算法的时间复杂度需要 O(N*K)。所幸没有使用额外的空间，空间的复杂度是O(1)。

不妨仔细看一下上面的图，内嵌的循环中，第一次和第二次都对 `12 + (-5) + -6` 进行了计算，也就是内层循环有着大量的重复计算。再次看上面的图，刨去重复计算的部分，后一次循环相对于前一次循环，都向右移动一格。就像一个窗口一样，往右边移动了一格。

使用双指针实现滑动窗口。窗口的范围为左闭右开的区间 `[lo, hi)` 。初始化 lo 和 hi 都为0， 窗口的大小为 hi - lo ，也是0。

由于题意可知，窗口最大值为 k（4），因此只要窗口尚未达到最大值，就右移一格，当窗口大小正好是 4 的时候，就说明原题存在一个解。一旦存在一个解，此时就可以将 lo 也向右移动一格，以实现缩小窗口。然后右边再继续滑动窗口，整个过程就像一个窗口向右滑动。一旦右边不能再滑动了，就直接缩小左边的窗口，直到窗口不合法返回结果。

移动的本质如下图：

![image.png](https://s2.ax1x.com/2020/01/09/lWzVy9.jpg)


* 初始窗口大小为 0 
* hi 向右移动一格，窗口大小为 1
* hi 再向右移动一格，窗口大小为 2
* hi 再向右移动一格，窗口大小为 3
* hi 再向右移动一个，窗口大小为 4，此时存在一个解。求解
* lo 向右一定一格，缩小窗口，窗口大小为3
* hi 再向右移动一格，窗口大小为 4， 此时又存在第二个解，求解

如此往复，直到 hi 到达末尾，lo 再缩小窗口就不合法，此时就结束算法。上面的算法兑换代码如下

In [3]:
class Solution:
    def findMaxAverage(self, nums: List[int], k: int) -> float:
        if not nums:
            return 0
        
        ret = float('-inf')
        # 初始化窗口计数器，用来判断窗口是否需要缩小
        windows = 0
        
        # 初始化窗口大小为 0
        lo = hi = 0     
         # 窗口未滑到最右 
        while hi < len(nums):      
            windows += nums[hi]         # 向右扩展一格
            hi += 1
            
            # 判断窗口计数器
            if hi - lo >= k:                      
                # 求解
                if hi - lo == k:                  
                    # 更新解 
                    ret = max(windows / k, ret)    
                # 缩小窗口
                windows -= nums[lo]    
                lo += 1                            
        return ret

通过窗口的滑动，尽管窗口再不停的变大或缩小，但是 hi 和 lo 都是亦步亦趋的往右边靠拢。整个过程算法的时间复杂度为 O(N)。

## 滑动窗口的模板

借助上面的题目可以一窥滑动窗口套路。滑动窗口的本质是窗口扩大的过程中，寻找一个解，然后窗口缩小的时候寻找一个最优解。如何理解呢？可以借助下面的一题

[209. 长度最小的子数组](https://leetcode-cn.com/problems/minimum-size-subarray-sum/)

> 给定一个含有 n 个正整数的数组和一个正整数 s ，找出该数组中满足其和 ≥ s 的长度最小的连续子数组。如果不存在符合条件的连续子数组，返回 0。
> 
> 示例: 
> 
> 输入: s = 7, nums = [2,3,1,2,4,3]
> 
> 输出: 2
> 
> 解释: 子数组 [4,3] 是该条件下的长度最小的连续子数组。


题意要求寻找一个连续的子数组，并且子数组的和要大于等于目标值s。这也是一个数组加和方式。回想上面的滑动窗口，窗口逐步扩大的时候。可以累加窗口的元素的和，如果大于目标s，那么就等于存在一个解。例如  [2, 3, 1, 2] 这个子数组，是hi 不停向右扩展。

有解并不代表最优解，假设上面第3个元素就是 7，  [2, 3, 1, 7] 自然是一个解，但是 单纯的 [7] 也是一个解。因此我们需要不断的缩小窗口，来找到最小窗口的解。代码如下：

In [4]:
class Solution:
    def minSubArrayLen(self, s: int, nums: List[int]) -> int:
        if not nums:
            return 0
        ret = len(nums) + 1
        lo, hi = 0, 0
        windows = 0
        while hi < len(nums):
            windows += nums[hi]
            hi += 1
            
            while windows >= s:
                ret = min(hi - lo, ret)
                windows -= nums[lo]
                lo += 1
        return ret if ret != len(nums) + 1 else 0

与前面一题类似，初始化窗口为 0，然后 hi 向右移动。每移动一格窗口后，检测当前的窗口计数器是否有解，有解即记录更新解，并且缩小窗口。

> NOTE 一般求最小子数组，有可能没有解，因此初始化解是数组本身的长度再+1， 方便最后判断是否有解的情况。

从上面的两个题大致可以看出，滑动窗口的套路无怪乎两步：

* 其一  向右滑动一格，
* 其二  滑动之后检测窗口计数器和题意要求，适当的缩小窗口

其伪代码可以表述如下：

```
lo = hi = 0
while hi < len(nums):
    windows.add(nums[hi])
    hi+ += 1

    while has_valid:
        handle(ret)
        windows.remove(s[lo])
        lo += 1
```

## 窗口计数器

滑动窗口的解题模板可以解决 leetcode 上大部分此类题目。然而算法之所以有魅力，一个重要原因就是不变中带有万变。有的算法稍微改变一下，其效果大相径庭。或者有的算法性能并不二至，有的写法可读性高，有的则讳莫苦涩。

在第二步骤中，通过窗口计数器与题意要求缩小窗口，也是求解的过程。另外一些题目，在缩小窗口的时候未必有解，而是缩小之后才有解。因此需要灵活的处理窗口计数器，缩小窗口，求解的过程。

geeksforgeeks 上有一题 `Longest Substring with K Distinct Characters`，即找出字串中不同字符数为K的最大子串


> 输入: String="araaci", K=2
> 
> 输出: 4
> 
> 解释: 不同字符数 K=2， 即有连个不同字符的最大子串  "araa".
> 
> 输入: String="araaci", K=1
> 
> 输出: 2
> 
> 解释: 不同字符数 K=1， 即有连个不同字符的最大子串  "aa".
> 
> 输入: String="cbbebi", K=3
> 
> 输出: 5
> 
> 解释: 不同字符数 K=3， 即有连个不同字符的最大串"cbbeb" & "bbebi".


由题意可知，求最长字串。使用一个窗口，窗口的不同字符数要等于 K。窗口的计数器可以设置当前窗口内不同字符的个数。然后接助一个hash表，用来记录当前窗口内的字符出现的次数。

窗口滑动的时候检查下一个字符是否出现，并更新不同字符数目。然后跟 k 进行比较，缩小窗口并求解。

滑动窗口对于求解字符出现次数，重复字符数，经常需要借助hash字典。后面会针对这样的case举例。

In [6]:
def longest_substring_with_k_distinct(str, k):
    ret = 0
    # 窗口计数器
    windows = 0
    windows_dct = dict()
    lo, hi = 0, 0

    while hi < len(str):
        windows_dct[str[hi]] = windows_dct.get(str[hi], 0) + 1
        if windows_dct[str[hi]] == 1:
            windows += 1
        hi += 1

        while windows > k:
            windows_dct[str[lo]] -= 1
            if windows_dct[str[lo]] == 0:
                windows -= 1
            lo += 1
        if windows == k:
            ret = max(hi - lo, ret)
    return ret

上面的代码也符合解题模板。窗口向右移动，然后检查移动之后，窗口扩大之后，当前的窗口计数器是否满足题意，即 窗口内不同的字符数 windows 是否大于 k，一旦大于 k，则缩小窗口。while 也可以写成 if 。因为
windows只要变更一次就不在合法。如果 windows 不大于 k，那么就是小于等于k。当等于k的时候，正好是题目的一个解，随即更新解即可。

尽管有内循环，最终的时间复杂度还是 O(N)。空间复杂度来自于存储 k 个字符的hash字典，即 O(K) 的空间复杂度

这个问题不同于解题模板之处在于，题解在判断的条件之外。即缩小窗口后才求解，此前一直是先求解再缩小窗口。然而问题殊途同归，谁先谁后并不一定，需要具体题目分析。这也是前文所说的算法变化的魅力。

同类题，leetcode上也有一道：

[904. 水果成篮](https://leetcode-cn.com/problems/fruit-into-baskets/)


> 在一排树中，第 i 棵树产生 tree[i] 型的水果。
> 你可以从你选择的任何树开始，然后重复执行以下步骤：
> 
> 把这棵树上的水果放进你的篮子里。如果你做不到，就停下来。
> 移动到当前树右侧的下一棵树。如果右边没有树，就停下来。
> 请注意，在选择一颗树后，你没有任何选择：你必须执行步骤 1，然后执行步骤 2，然后返回步骤 1，然后执行步骤 2，依此类推，直至停止。
> 
> 你有两个篮子，每个篮子可以携带任何数量的水果，但你希望每个篮子只携带一种类型的水果。
> 用这个程序你能收集的水果总量是多少？
> 
> 示例 1：
> 
> 输入：[1,2,1]
> 
> 输出：3
> 
> 解释：我们可以收集 [1,2,1]。
> 
> 示例 2：
> 
> 输入：[0,1,2,2]
> 
> 输出：3
> 
> 解释：我们可以收集 [1,2,2].
> 
> 如果我们从第一棵树开始，我们将只能收集到 [0, 1]。
> 
> 示例 3：
> 
> 输入：[1,2,3,2,2]
> 
> 输出：4
> 
> 解释：我们可以收集 [2,3,2,2].
> 
> 如果我们从第一棵树开始，我们将只能收集到 [1, 2]。
> 
> 示例 4：
> 
> 输入：[3,3,3,1,2,1,1,2,3,3,4]
> 
> 输出：5
> 
> 解释：我们可以收集 [1,2,1,1,2].
> 
> 如果我们从第一棵树或第八棵树开始，我们将只能收集到 4 个水果。

也许是为了增加解题的趣味性，题目和现实生活相结合。然而解题的过程中要学会挖掘抽象题意。

尽管有两个篮子，但是篮子里只能有同一种水果。如果把两个篮子放在一起。就和前一题类似了。两个篮子组成一个数组。数组中只能出现 2 次不同的水果（k=2）。所以使用滑动窗口解法如下；

In [7]:
class Solution:
    def totalFruit(self, tree: List[int]) -> int:
        ret = 0
        lo, hi = 0, 0
        windows = 0
        windows_dct = dict()
        while hi < len(tree):
            windows_dct[tree[hi]] = windows_dct.get(tree[hi], 0) + 1
            if windows_dct[tree[hi]] == 1:
                windows += 1
            hi += 1
            size = hi - lo

            while windows > 2:
                windows_dct[tree[lo]] -= 1
                if windows_dct[tree[lo]] == 0:
                    windows -= 1
                lo += 1

            if windows <= 2:
                ret = max(hi - lo, ret)
        return ret

## 滑动窗口的解

正确的设置滑动计数器，以便在合适的时候缩小窗口。直接关系到滑动窗口的解。正如 水果成篮 这类题一样，需要花点心思将题目抽象成滑动窗口的类型，然后才能针对性的进行求解。下面再看两个这样的转换思路题目

[1004. 最大连续1的个数 III](https://leetcode-cn.com/problems/max-consecutive-ones-iii/)

> 给定一个由若干 0 和 1 组成的数组 A，我们最多可以将 K 个值从 0 变成 1 。
> 返回仅包含 1 的最长（连续）子数组的长度。
> 
> 示例 1：
> 
> 输入：A = [1,1,1,0,0,0,1,1,1,1,0], K = 2
> 
> 输出：6
> 
> 解释： 
> 
> [1,1,1,0,0,1,1,1,1,1,1]
> 
> 粗体数字从 0 翻转到 1，最长的子数组长度为 6。
> 
> 示例 2：
> 
> 输入：A = [0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,1,1,1,1], K = 3
> 
> 输出：10
> 
> 解释：
> 
> [0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1]
> 
> 粗体数字从 0 翻转到 1，最长的子数组长度为 10

题意很明确，将 k 个 0 替换成 1，替换后的字串中连续的 1 最长。由于替换后，再去检查串的连续性，这是一种思路。但是对于继续右移的时候，替换的串该如何处理？

一个转换思路就是，记录当前连续的 1 的个数，窗口的大小 减去 连续的个数，就是剩余可以替换的个数，如果这个个数等于 k ，那么正好是一个解。因此代码如下：

In [8]:
class Solution:
    def longestOnes(self, A: List[int], K: int) -> int:
        if not A:
            return 0
        
        ret = 0
        lo = hi = 0

        max_1_repeat_size = 0
        
        while hi < len(A):
            if A[hi] == 1:
                max_1_repeat_size += 1
                
            hi += 1            
            if hi - lo - max_1_repeat_size > K:
                if A[lo] == 1:
                    max_1_repeat_size -= 1
                lo += 1
            
            ret = max(hi - lo, ret)
        return ret

这一题的关键是如何找到滑动窗口的解。通常情况下，都会利用连续和匹配这样的转换技巧。解决了这一题，下面的一题也就迎刃而解了

[424. 替换后的最长重复字符](https://leetcode-cn.com/problems/longest-repeating-character-replacement/)


> 给你一个仅由大写英文字母组成的字符串，你可以将任意位置上的字符替换成另外的字符，总共可最多替换 k 次。在执行上述操作后，找到包含重复字母的最长子串的长度。
> 
> 注意:
> 字符串长度 和 k 不会超过 104。
> 
> 示例 1:
> 
> 输入: s = "ABAB", k = 2
> 输出: 4
> 解释:
> 用两个'A'替换为两个'B',反之亦然。
> 示例 2:
> 
> 输入: s = "AABABBA", k = 1
> 输出:4
> 解释: 将中间的一个'A'替换为'B',字符串变为 "AABBBBA"。子串 "BBBB" 有最长重复字母, 答案为 4。

这到题的变化之处在于不再是求某个给定的可以替换的字符，而是串中的所有字符都可以替换。万变不离其中，依然可以找到当前窗口中最长的重复字串，然后将其它字符变更为当前重复的字符即可。如何查找重复字符呢？解法稍后

## 滑动窗口遇到哈希

leetcode上有一类题，所求的字串除了最大最小之外，还有是否重复，模式匹配的题目。尤其是后一种，通常以 hard 题目出现。

对于字符是否重复或者是否出现过，可以借助一个 hash 字典结构存储器出现的次数，而窗口计数器也可以使用另外一个 hash 字典存储所需要对比的数据。

对于 `替换后的最长重复字符` 这题，我们可以创建一个 hash 字典，又来存储字符出现的次数。窗口向右移动的时候，更新窗口字符的计数，求出最长重复的字符和长度。剩下的套路就和之前翻转 1 的类似了。具体代码如下：

In [9]:
class Solution:
    def characterReplacement(self, s: str, k: int) -> int:
        ret = 0
        lo = hi = 0
        
        t = dict()
        max_repeat = 0
        
        while hi < len(s):
            t[s[hi]] = t.get(s[hi], 0) + 1
            max_repeat = max(max_repeat, t[s[hi]]) 
            hi += 1
            
            if hi - lo - max_repeat > k:
                t[s[lo]] -= 1
                lo += 1
            ret = max(hi - lo, ret)
        return ret


找重复字符，或者不同字符，往往需要借助 hash 字典来存储字符出现的次数。Leetcode 第三题比较经典

[3. 无重复字符的最长子串](https://leetcode-cn.com/problems/longest-substring-without-repeating-characters/)


> 给定一个字符串，请你找出其中不含有重复字符的 最长子串 的长度。
> 
> 示例 1:
> 
> 输入: "abcabcbb"
> 输出: 3 
> 解释: 因为无重复字符的最长子串是 "abc"，所以其长度为 3。
> 示例 2:
> 
> 输入: "bbbbb"
> 输出: 1
> 解释: 因为无重复字符的最长子串是 "b"，所以其长度为 1。
> 示例 3:
> 
> 输入: "pwwkew"
> 输出: 3
> 解释: 因为无重复字符的最长子串是 "wke"，所以其长度为 3。
>      请注意，你的答案必须是 子串 的长度，"pwke" 是一个子序列，不是子串。

题意要求的是寻找最长的无重复的字串。最长可以理解为窗口最大的时候的解。无重复如何判断呢？首先思考如何判断一个字串是否有重复字符。一个简单的方法就是迭代字符串，依次数这些字符出现的次数。如果数到大于 1，那么肯定就有重复的字符。

因此反向思考，如果遇到一个字符，在字符表里没有出现，那么就是没有重复字符。更进一步，使用滑动窗口的时候，hi 向右滑动的时候必然会读到一个字符，如果这个字符没有重复，那么hi可以继续滑动。如果这个字符有重复，那么就说明需要缩小窗口了，直到窗口内没有重复的字符。

对于 字串 `abcabcbb` , 只有当 hi 为第四个字符的时候，才出现重复字符 `a`，此时就需要缩小窗口，即 lo 也向右移动。移动多少次呢？因为是右边字符率先重复的，重复的字符未必是左边当前 lo 的字符，因此lo 需要逐步缩小，直到找到那个重复的 hi 字符。下图可以清晰的说明这种情况。

![图片](https://s2.ax1x.com/2020/01/09/lWzMFK.jpg)


当 a 重复的时候，lo为0， lo 需要不断的右移，直到把 a 剔除在外。最后附上代码如下

In [10]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        ret = 0
        lo, hi = 0, 0
        windows_dct = dict()
        while hi < len(s):
            windows_dct[s[hi]] = windows_dct.get(s[hi], 0) + 1
            hi += 1
            
            while windows_dct[s[hi - 1]] > 1:
                windows_dct[s[lo]] = windows_dct[s[lo]] - 1
                lo += 1
            ret = max(hi - lo, ret)
        return ret

使用一个 windows 字典用来记录字符出现的次数。hi 右移的时候，更新 hi 字符的次数。随即判断这个 字符 是否重复，如果重复，则需要缩小窗口。一旦窗口不需要缩小，表面有一个字串肯定不重复，即存在一个解。此时更新这个解即可。经过窗口的滑动，最后可以把所有的解求出，并且在动态求解的过程中只保存最优解。

当然，上面一题的滑动解法还可以写成下面的样子:

In [11]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        if not s:
            return 0
        
        ret = 0
        i = j = 0
        t = dict()
        while i < len(s):   
            if j < len(s) and t.get(s[j], 0) == 0:
                t[s[j]] = 1
                j += 1
            else:
                t[s[i]] -= 1
                i += 1
            ret = max(j - i, ret)
        return ret

思路一样，但是和之前介绍的解题模板稍有不同。因此不再详细解释。

最后，上面的两种解法，都使用了一个hash字典来做计数器。使用的额外的空间，其空间复杂度为 O(K)，K 为重复字符的个数，最坏的情况下 K <= N 。当然对于 英文字母，可以使用一个固定大小的数组来存储器出现的频次。时间复杂度依然O(N)。

## 滑动窗口与匹配

前面所见的题目，多半是处理某个串或数组在某个条件下的某种特性的解。leetcode还有一类题，给出两个串，其中一个和之前的一样，另外一个则是一个 模式串 pattern。需要 字串在模式串的某些条件约束下求解问题。这类的题目的复杂度初看会比上面的类型要难。但是，只要掌握好模式匹配的技巧，依然可以轻松破解。

[567. 字符串的排列](https://leetcode-cn.com/problems/permutation-in-string/)


> 给定两个字符串 s1 和 s2，写一个函数来判断 s2 是否包含 s1 的排列。
> 换句话说，第一个字符串的排列之一是第二个字符串的子串。
> 
> 示例1:
> 
> 输入: s1 = "ab" s2 = "eidbaooo"
> 输出: True
> 解释: s2 包含 s1 的排列之一 ("ba").
 > 
> 示例2:
> 
> 输入: s1= "ab" s2 = "eidboaoo"
> 输出: False

题意要求 s1的字串可以任意重新组合，组合之后，可以在 s2 中匹配。假如说A串完全匹配 B串，那么意味着A串和B串一模一样，字母的顺序和次数都应该一致。s1 匹配 s2 的字串，也就是说需要在 s2 中，找到s1出现的字母，并且次数也要一致。同时，s2 的匹配的字串，其长度也要等于 s1。

了解这两个限制之后，使用上述 hash 字典的技巧，可以先求出一个字符表。即 s1 所有字符出现的次数。然后设置一个滑动窗口。窗口右移的过程中，判断右移的字符是否匹配了s1串对应的字符（次数一致），一旦窗口的字符都匹配了s1字符。那么就可以进行窗口的缩小了。此时，窗口很可能存在一个解。正如上面所说，匹配要长度也一致，即窗口的大小如果等于 s1串的长度，那么就存在这一一个解。上面的过程可以用下图所示：

![image.png](https://s2.ax1x.com/2020/01/09/lWzGyd.jpg)

当 窗口右移的时候，记录 hi 的字符出现的次数。同时如果 出现了 s1 中的字符数一致，则表明该字符匹配了s1。对于上图的第二个数组，当窗口的所有字符都匹配了 s1 ，此时 hi - lo ，窗口大小为 5，显然跟 s1 的长度不匹配，因此需要缩小窗口。只有当窗口匹配s1，并且 hi - lo 的长度正好等于 s1 的长度，才是最终的解。

当然，在缩小窗口的时候，如果 a和b 不是相邻的元素，那么最终在缩小窗口的时候，导致了 窗口中的字符不再和 s1 中的字符匹配，此时就是没有解的情况。

需要一个hash字典（need_dct）记录 s1 的字符数，同时另外一个 hash 字典(windows_dct)记录窗口字符数，还需要一个是否匹配的match 变量，用来标记 窗口中字符在 s1中的匹配情况。窗口计数器可以看成是 windows_dct 和 match，因为在窗口变更的时候，都可能需要修改这俩个量。

最终算法兑换为代码如下

In [12]:
class Solution:
    def checkInclusion(self, s1: str, s2: str) -> bool:
        need_dct = {}
        for i in s1:
            need_dct[i] = need_dct.get(i, 0) + 1
        
        windows_dct = {}
        ret = False
        lo = hi = 0
        
        match = 0
        while hi < len(s2):
            hi_char = s2[hi]
            windows_dct[hi_char] = windows_dct.get(hi_char, 0) + 1
            if windows_dct[hi_char] == need_dct.get(hi_char, 0):
                match += 1
            hi += 1

            while match == len(need_dct):
                if hi - lo == len(s1):     # 字符数匹配，长度也要匹配，不能出现别的字符
                    return True
                
                lo_char = s2[lo]
                windows_dct[lo_char] -= 1
                if windows_dct[lo_char] < need_dct.get(lo_char, 0):
                    match -= 1
                lo += 1
        return ret


对于向右移动的 hi_char ，可以加一层判断，即 只有 hi_char 是 need_dct 的字母，才更新 windows_dct ，这样会节省很多空间。由于为了说明算法思路，上面的写法就不针对这种情况进行优化。

从中可以看出，hash 字典计数的方式前面出现过多次。match 这个变量的设计是另外一个关键，当 s1 中有重复的字符的时候，缩小窗口尤其要注意。增加 match的时候是一个 `=`，减少 match 的时候是一个 `<` 号，具体逻辑想清楚之后也很好理解。

算法的时间复杂度来自两个串的处理，即 O(N+K)， 空间复杂度如果只存模式串的字符，那么可以优化成O(K)。

## 匹配与包含

匹配有着严格的定义。而包含就松了很多。匹配的求解是窗口大小要严格等于模式串的长度。而包含就未必，只需要窗口字符的次数跟模式串的次数一致即可。leetcode 有一到 hard的题目，就是一种包含关系。

[76. 最小覆盖子串](https://leetcode-cn.com/problems/minimum-window-substring/)


> 给你一个字符串 S、一个字符串 T，请在字符串 S 里面找出：包含 T 所有字母的最小子串。
> 
> 示例：
> 
> 输入: S = "ADOBECODEBANC", T = "ABC"
> 输出: "BANC"

题目的长度很短，难道却不小。当然掌握了滑动技巧，难道就更小了。与上题一样，只要找到match的模式串，然后再缩小窗口即可。

In [13]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        if not t:
            return ''

        need_dct = {}
        for i in t:
            need_dct[i] = need_dct.get(i, 0) + 1

        ret_size = len(s) + 1
        ret = None
        lo = hi = 0
        windows_dict = dict()

        match = 0
        while hi < len(s):
            if s[hi] in need_dct:
                windows_dict[s[hi]] = windows_dict.get(s[hi], 0) + 1
                if windows_dict[s[hi]] == need_dct[s[hi]]:
                    match += 1
            hi += 1
            
            while match == len(need_dct):
                if hi - lo < ret_size:
                    ret_size = hi - lo
                    ret = s[lo:hi]
                if s[lo] in windows_dict:
                    windows_dict[s[lo]] -= 1
                    if windows_dict[s[lo]] < need_dct[s[lo]]:
                        match -= 1
                lo += 1
        if ret_size != len(s) + 1:
            return ret
        return ''

由于是求最小子串，求解过程中，需要更新解。上面使用了 字串的切片更新解，可以优化中间存储的是解的 lo 和 hi，最后返回的时候再切片。

输入有两个串，滑动窗口的串为 N，滑动过程中的时间复杂度为 O(N)，模式串hash字符表构建的时候至少也需要遍历一次，因此算法最终复杂度是 O(N+K)

空间方面，也是需要 hash 用来进行窗口计数器，其复杂度是O(K)，而对于中间结果的存储，上面的方式最坏需要 O(N)的空间来存储解。


这类问题还有一道题可以用来练习 [438. 找到字符串中所有字母异位词](https://leetcode-cn.com/problems/find-all-anagrams-in-a-string/)

如果对之前的滑动套路比较了解了，解这题就有点老生常谈了。前面的题目要求求最小或者最大。这道题则是要求所有解。

> 给定一个字符串 s 和一个非空字符串 p，找到 s 中所有是 p 的字母异位词的子串，返回这些子串的起始索引。
> 字符串只包含小写英文字母，并且字符串 s 和 p 的长度都不超过 20100。
> 
> 说明：
> 
> 字母异位词指字母相同，但排列不同的字符串。
> 不考虑答案输出的顺序。
> 示例 1:
> 
> 输入: s: "cbaebabacd" p: "abc"
> 
> 输出: [0, 6]
> 解释:起始索引等于 0 的子串是 "cba", 它是 "abc" 的字母异位词。起始索引等于 6 的子串是 "bac", 它是 "abc" 的字母异位词。
> 示例 2:
> 
> 输入:s: "abab" p: "ab"
> 
> 输出:[0, 1, 2]
> 解释:起始索引等于 0 的子串是 "ab", 它是 "ab" 的字母异位词。
> 起始索引等于 1 的子串是 "ba", 它是 "ab" 的字母异位词。
> 起始索引等于 2 的子串是 "ab", 它是 "ab" 的字母异位词。


题目是 medium 类型，但是通过率却很低，大概很多人尝试了一下，然后提交才发现遗漏不少注意点。这道和模式匹配的几乎一样。只是在求解的过程中存储多个解罢了。下面直接给出代码

In [14]:
class Solution:
    def findAnagrams(self, s: str, p: str) -> List[int]:
        
        ret = []
        need_dct = {}
        for i in p:
            need_dct[i] = need_dct.get(i, 0) + 1
            
        windows_dct = {}
        lo = hi = 0
        match = 0
        
        while hi < len(s):
            windows_dct[s[hi]] = windows_dct.get(s[hi], 0) + 1
            if windows_dct[s[hi]] == need_dct.get(s[hi], -1):
                match += 1
            hi += 1
                           
            while match == len(need_dct):
                if hi - lo == len(p):
                    ret.append(lo)
                windows_dct[s[lo]] -= 1
                if windows_dct[s[lo]] < need_dct.get(s[lo], -1):
                    match -= 1
                lo += 1
                           
        return ret


滑动窗口的解题模板基本使用。老生常谈的两步，第一步右移窗口，更新窗口计数器。第二步 通过窗口计数器缩小窗口，寻找解，更新解，更新窗口计数器。


## 字符与单词的滑动
    
此前的滑动窗口都是字符的滑动，还有一类是单词的滑动。这类题目的难道一下加大了。当然，对于这类问题，单词滑动本身也没有太多可圈可点的技巧。比较麻烦的在单词的起始位置不再是字串的第一个字符了。例如 s[0:len(s)] 可以进行单词滑动，s[1:len(s)] 也可以进行单词滑动。这类题滑动窗口就不算是一个比较好的解法。

[30. 串联所有单词的子串](https://leetcode-cn.com/problems/substring-with-concatenation-of-all-words/)


> 注意子串要与 words 中的单词完全匹配，中间不能有其他字符，但不需要考虑 words 中单词串联的顺序。
> 示例 1：
> 
> 输入：s = "barfoothefoobarman",
>   words = ["foo","bar"]
> 输出：[0,9]
> 解释：
> 从索引 0 和 9 开始的子串分别是 "barfoo" 和 "foobar" 。
> 输出的顺序不重要, [9,0] 也是有效答案。
> 示例 2：
> 
> 输入：
>   s = "wordgoodgoodgoodbestword",
>   words = ["word","good","best","word"]
> 输出：[]

题目给出的模式数组，其中元素是一个三个字符的单词，如果问题是 'bftfbm' 中串联 f 和 b，问题就和滑动窗口很似了。如果是 `barfoothefoobarman` 和 `foo bar` ，我们可以把 hi 和 lo 的步长设置为 单词的字符数大小。即一次向右移动三格。

![image.png](https://s2.ax1x.com/2020/01/09/lWzdFf.jpg)

In [15]:
# 错误答案
class Solution:

    def findSubstring(self, s: str, words: List[str]) -> List[int]:
        if len(words) == 0 or len(words[0]) == 0:
            return []

        need_dct = {}
        for w in words:
            need_dct[w] = need_dct.get(w, 0) + 1

        words_count = len(words)
        words_len = len(words[0])

        ret = []
        
        lo = hi = 0
        match = 0
        windows_dct = {}
        while hi < len(s):
            windows_dct[s[hi:hi+words_len]] = windows_dct.get(s[hi:hi+words_len], 0) + 1
            if windows_dct[s[hi:hi+words_len]] == need_dct.get(s[hi:hi+words_len], -1):
                match += 1
            hi += words_len
            
            while match == len(need_dct):
                if hi - lo == words_len * words_count:
                    ret.append(lo)
                windows_dct[s[lo:lo+words_len]] -= 1
                if windows_dct[s[lo:lo+words_len]] < need_dct.get(s[lo:lo+words_len], -1):
                    match -= 1
                lo += words_len
                
        return ret

很不幸，上面的代码，对于 `barfoothefoobarman` 和 `wordgoodgoodgoodbestword` 都能正确的返回，其实也只是一种巧合罢了。对于 `abarfoothefoobarman` 的输入，则会求解错误。原因也很简单，单词是按照 单词的长度进行跳跃的，对于 后面字串的起始字符是 a，也就是原字符的 起始字符 b 将不会有机会成为起始字符。

解决的思路也很简单，让原始输入的字串的每个字符都成为一次起始字符，再使用上面滑动窗口作为子函数求解即可。

还是不够走运，这样的提交，本身解法没有错，但是运行的时间太长，leetcode会拒绝。怎么办呢？此时的问题恰恰是滑动窗口的问题。每个字符作为一次起始，这个过程看了是无法去除的。剩下可以优化的就是滑动窗口。

由题意已知，起始知道每次需要匹配判断字串长度（words_len * words_count），最外层的循环是可以提前结束的。此外，在滑动窗口内部，一旦不匹配，也是可以提前返回的。因此修改之后的代码如下：

In [16]:
class Solution:

    def findSubstring(self, s: str, words: List[str]) -> List[int]:
        if len(words) == 0 or len(words[0]) == 0:
            return []

        need_dct = {}
        for w in words:
            need_dct[w] = need_dct.get(w, 0) + 1

        words_count = len(words)
        words_len = len(words[0])

        ret = []
        for i in range((len(s) - words_count * words_len) + 1):
            r = self._find_sub_string(i, s, words_len, words_len * words_count, need_dct)
            if r != -1:
                ret.append(r)
        return ret

    def _find_sub_string(self, start, s, words_len, sub_string_len, need_dct):

        lo = hi = start
        windows_dct = {}
        match = 0
        while hi < start + sub_string_len:
            if not need_dct.get(s[hi:hi+words_len]):
                return -1

            windows_dct[s[hi:hi+words_len]] = windows_dct.get(s[hi:hi+words_len], 0) + 1
            if windows_dct[s[hi:hi+words_len]] > need_dct.get(s[hi:hi+words_len], -1):
                return -1
            hi += words_len

        if hi - lo == sub_string_len:
            return lo
        else:
            return -1

虽然使用了滑动窗口的思路，但是不再是像之前那样亦步亦趋了。这个解法最终可以跑过 leetcode。实际上还是耗时比较长，算不上一个好的解法。

解法的时间复杂度是 O(N * K * Len) ，N 是字串的长度，K 是单词的数目，Len 则是每个单词的长度。由此可见，当 K 和 len 比较小的时候，算法还可以。当 K 和 N 差不多的时候，其复杂度就比较高了。

## 总结

综上所述，滑动窗口套路对于一些求解连续的字串子数组的问题非常有用。滑动窗口的思路就是设置一个再字符串或数组上进行滑动。窗口扩大的时候寻找一个解，窗口缩小的时候寻找最优解。滑动窗口解法的模板就比较死板。先将窗口不断右移，每次移动之后，使用窗口计算器判断解，判断窗口是否可以缩小。然后再循环往复。

这里的关键是如何定义窗口计数器和找出解。或者如何将一个问题抽象成滑动窗口问题。需要不断的进行练习和总结。

另外，在解决一些重复，匹配模式的字串问题，可以借助hash字典记录字符的出现次数。这些hash字典通常也可以成为窗口计数器来求解问题。

## 后记

文中的例题代码，经过了 leetcode 的提交测试并且通过了。为了演示滑动窗口的套路，代码实现尽量写起来像模板一样。实际上，只要思路类似，写法和实现未必需要像模板那么死板。

如很多情况下 for in 迭代在 python中都可以比 while 循环更优雅（至少不会因忘记写 hi += 1这样的条件造成死循环）。总而言之，熟练掌握滑动窗口的思想，可以写出优雅代码，而不是死套模板。

> [643. 子数组最大平均数 I](https://leetcode-cn.com/problems/maximum-average-subarray-i/)
>
> [209. 长度最小的子数组](https://leetcode-cn.com/problems/minimum-size-subarray-sum/)
>
> [904. 水果成篮](https://leetcode-cn.com/problems/fruit-into-baskets/)
>
> [1004. 最大连续1的个数 III](https://leetcode-cn.com/problems/max-consecutive-ones-iii/)
>
> [424. 替换后的最长重复字符](https://leetcode-cn.com/problems/longest-repeating-character-replacement/)
>
> [3. 无重复字符的最长子串](https://leetcode-cn.com/problems/longest-substring-without-repeating-characters/)
>
> [567. 字符串的排列](https://leetcode-cn.com/problems/permutation-in-string/)
>
> [76. 最小覆盖子串](https://leetcode-cn.com/problems/minimum-window-substring/)
>
> [438. 找到字符串中所有字母异位词](https://leetcode-cn.com/problems/find-all-anagrams-in-a-string/)
>
> [30. 串联所有单词的子串](https://leetcode-cn.com/problems/substring-with-concatenation-of-all-words/)
>
> 来源：力扣（LeetCode） [https://leetcode-cn.com/problemset/all/](https://leetcode-cn.com/problemset/all/)